In [2]:
import pandas as pd
import datetime
import ccxt

1. Data Collection

In [3]:
print(ccxt.exchanges)

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitforex', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitvavo', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcmarkets', 'btctradeua', 'btcturk', 'bybit', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinmate', 'coinone', 'coinsph', 'coinspot', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'fmfwio', 'gate', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'lbank2', 'luno', 'lykke', 'mercado', 'mexc', 'mexc3', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okex', 'okex5', 'okx', 'paymium'

In [4]:
exchange = ccxt.binance()

In [5]:
# Retrieve trading pairs
markets = exchange.load_markets()

# Create a DataFrame from the list of trading pairs
df_currencies = pd.DataFrame(list(markets.values()))

In [6]:
df_currencies['type'].value_counts()

spot      2417
swap       295
future      36
Name: type, dtype: int64

In [7]:
# Delete all data that are not quoted upon USD as well as keep only swap data points
df_currencies = df_currencies[df_currencies['quote'] == 'USD']
df_currencies = df_currencies[df_currencies['type'] == 'swap']

# Reset index
df_currencies = df_currencies.reset_index(drop=True)

In [8]:
df_currencies['id'].describe()

count              41
unique             41
top       BTCUSD_PERP
freq                1
Name: id, dtype: object

In [9]:
df_currencies.to_excel('Currencies.xlsx')

Retrieval of Data in a set period

Issue in the code.. period definition is not working properly

In [10]:
# Define a list of trading pairs
symbols = ['ETH/USD']

# Specify the timeframe
timeframe = '5m'

start_date = exchange.parse8601('2023-01-01T00:00:00Z')  # Start date in ISO8601 format
end_date = exchange.parse8601('2023-12-28T00:00:00Z')  # End date in ISO8601 format

limit = 1000  # Number of data points to fetch per request

# Create an empty DataFrame to store the data
df = pd.DataFrame()

import time  # Import the time module for sleep

while start_date < end_date:
    data_available = False  # Flag to check if data is available for at least one symbol
    max_timestamp = None  # To keep track of the maximum timestamp
    
    for symbol in symbols:
        # Calculate the end timestamp for this data request
        end_timestamp = min(start_date + limit * 300 * 1000, end_date)
        
        # Fetch the OHLCV data for each trading pair within the specified date range
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, start_date, limit, params={'endTime': end_timestamp})
        
        if ohlcv:
            data_available = True  # Data is available for at least one symbol
            
            # Extract the 'Open' and 'Volume' values and store them in a DataFrame
            open_values = [candle[1] for candle in ohlcv]
            volume_values = [candle[5] for candle in ohlcv]
            
            # Create a DataFrame for the current pair
            df_pair = pd.DataFrame({
                'Timestamp': [candle[0] for candle in ohlcv],
                f'{symbol}_Open': open_values,  # Rename the open column
                f'{symbol}_Volume': volume_values  # Create a new volume column
            })
            
            if df.empty:
                df = df_pair
            else:
                # Merge the DataFrames on the 'Timestamp' column with a concat
                df = pd.concat([df, df_pair], ignore_index=True, sort=False)
            
            # Update the maximum timestamp for all pairs
            max_symbol_timestamp = ohlcv[-1][0]
            if max_timestamp is None or max_symbol_timestamp > max_timestamp:
                max_timestamp = max_symbol_timestamp
        
    # Update the start time for the next request
    start_date = max_timestamp + 1  # Increment by the maximum timestamp
    
    # Check if there are more data points to fetch, if not, exit the loop
    if not data_available or start_date >= end_date:
        break
    
    # Introduce a small delay to avoid hitting rate limits
    time.sleep(1)


# Sort the DataFrame by 'Timestamp'
df.sort_values(by='Timestamp', inplace=True)

# Convert the 'Timestamp' column to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')

# Reset the index
df.reset_index(drop=True, inplace=True)

# Export the DataFrame to an Excel file
df.to_excel('Pair Data1.xlsx')

df

,Timestamp,ETH/USD_Open,ETH/USD_Volume
0,2023-01-01 00:05:00,1195.67,771.746245
1,2023-01-01 00:10:00,1194.62,561.227518
2,2023-01-01 00:15:00,1193.84,990.491806
3,2023-01-01 00:20:00,1195.29,151.990161
4,2023-01-01 00:25:00,1195.53,231.066821
...,...,...,...
103963,2023-12-27 23:40:00,2378.87,1442.798922
103964,2023-12-27 23:45:00,2384.91,1355.289090
103965,2023-12-27 23:50:00,2380.35,1583.140458
103966,2023-12-27 23:55:00,2380.15,606.417749


2. Feature Engineering

- Create new features that might be relevant for the trading strategy. For example, moving averages, relative strength index (RSI), moving average convergence divergence (MACD), and other technical indicators.
- Consider adding lag features, such as the previous period's prices or volumes, as they might provide useful information.

In [11]:

# 1. Feature Engineering - Moving Averages
df['ETH/USD_MA_10'] = df['ETH/USD_Open'].rolling(window=10).mean()
df['ETH/USD_MA_30'] = df['ETH/USD_Open'].rolling(window=30).mean()

# 2. Feature Engineering - Relative Strength Index (RSI)
def calculate_rsi(data, window=14):
    diff = data.diff(1)
    gain = diff.where(diff > 0, 0)
    loss = -diff.where(diff < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['ETH/USD_RSI'] = calculate_rsi(df['ETH/USD_Open'])

# 3. Feature Engineering - Moving Average Convergence Divergence (MACD)
short_window = 12
long_window = 26

df['ETH/USD_Short_MA'] = df['ETH/USD_Open'].rolling(window=short_window).mean()
df['ETH/USD_Long_MA'] = df['ETH/USD_Open'].rolling(window=long_window).mean()

df['ETH/USD_MACD'] = df['ETH/USD_Short_MA'] - df['ETH/USD_Long_MA']

# Calculate the 9-period EMA of the MACD
macd_period = 9

# Calculate the MACD line (difference between short-term and long-term EMAs)
df['ETH/USD_MACD'] = df['ETH/USD_Short_MA'] - df['ETH/USD_Long_MA']

# Calculate the signal line (9-period EMA of the MACD)
df['ETH/USD_Signal_Line'] = df['ETH/USD_MACD'].ewm(span=macd_period, adjust=False).mean()

In [14]:
# Define a window for looking back and looking forward
lookback_window = 5  # You may adjust these values based on your strategy
lookforward_window = 5

# Create a new column for the target variable
df['Target_Variable'] = 0  # Initialize with a default value

# Determine buy signals (bottoms)
df.loc[
    (df['ETH/USD_Open'] < df['ETH/USD_Open'].shift(1).rolling(window=lookback_window).min()) &
    (df['ETH/USD_Open'] < df['ETH/USD_Open'].shift(-1).rolling(window=lookforward_window).min()),
    'Target_Variable'
] = 1  # 1 represents a buy signal

# Determine sell signals (tops)
df.loc[
    (df['ETH/USD_Open'] > df['ETH/USD_Open'].shift(1).rolling(window=lookback_window).max()) &
    (df['ETH/USD_Open'] > df['ETH/USD_Open'].shift(-1).rolling(window=lookforward_window).max()),
    'Target_Variable'
] = -1  # -1 represents a sell signal

# Print debug information
print("Buy Conditions:")
print(df[df['Target_Variable'] == 1][['Timestamp', 'ETH/USD_Open']])

print("\nSell Conditions:")
print(df[df['Target_Variable'] == -1][['Timestamp', 'ETH/USD_Open']])

df.to_excel("Features1.xlsx", index=False)


Buy Conditions:
Empty DataFrame
Columns: [Timestamp, ETH/USD_Open]
Index: []

Sell Conditions:
Empty DataFrame
Columns: [Timestamp, ETH/USD_Open]
Index: []


In [ ]:
# Define a window for looking back and looking forward
lookback_window = 10  # You may adjust these values based on your strategy
lookforward_window = 10

# Create a new column for the target variable
df['Target_Variable'] = 0  # Initialize with a default value

# Determine buy signals (bottoms)
df['Buy_Signal'] = (
    (df['ETH/USD_Open'] < df['ETH/USD_Open'].shift(1).rolling(window=lookback_window).min()) &
    (df['ETH/USD_Open'] < df['ETH/USD_Open'].shift(-1).rolling(window=lookforward_window).min())
)

# Determine sell signals (tops)
df['Sell_Signal'] = (
    (df['ETH/USD_Open'] > df['ETH/USD_Open'].shift(1).rolling(window=lookback_window).max()) &
    (df['ETH/USD_Open'] > df['ETH/USD_Open'].shift(-1).rolling(window=lookforward_window).max())
)


# Print the count of buy and sell signals
print("Buy Signals Count:", df['Buy_Signal'].sum())
print("Sell Signals Count:", df['Sell_Signal'].sum())

# Set the target variable based on buy and sell signals
df.loc[df['Buy_Signal'], 'Target_Variable'] = 1  # 1 represents a buy signal
df.loc[df['Sell_Signal'], 'Target_Variable'] = -1  # -1 represents a sell signal

# Drop the temporary Buy_Signal and Sell_Signal columns if needed
# df.drop(['Buy_Signal', 'Sell_Signal'], axis=1, inplace=True)

df.to_excel("Features1.xlsx")

Buy Signals Count: 0
Sell Signals Count: 0


In [ ]:
df.columns

Index(['Timestamp', 'ETH/USD_Open', 'ETH/USD_Volume', 'ETH/USD_MA_10',
       'ETH/USD_MA_30', 'ETH/USD_RSI', 'ETH/USD_Short_MA', 'ETH/USD_Long_MA',
       'ETH/USD_MACD', 'ETH/USD_Signal_Line', 'Target_Variable'],
      dtype='object')

3. Backtesting

In [ ]:
initial_capital = 100000  # Set your initial capital
cash = initial_capital
position = 0  # Number of units held
portfolio_value = []  # To store portfolio value at each timestamp

for index, row in df.iterrows():
    # Check for Buy Signal
    if row['Buy_Signal']:
        if cash > 0:
            # Buy the asset using available cash
            position = cash / row['ETH/USD_Open']
            cash = 0
            print(f"Buying at {row['Timestamp']} - Price: {row['ETH/USD_Open']}, Position: {position}")

    # Check for Sell Signal
    elif row['Sell_Signal']:
        if position > 0:
            # Sell the entire position
            cash = position * row['ETH/USD_Open']
            position = 0
            print(f"Selling at {row['Timestamp']} - Price: {row['ETH/USD_Open']}, Cash: {cash}")

    # Calculate portfolio value at each timestamp
    portfolio_value.append(cash + position * row['ETH/USD_Open'])

# Add portfolio_value to the DataFrame
df['Portfolio_Value'] = portfolio_value


KeyError: 'Buy_Signal'

The choice of a time frame for a momentum trading bot depends on your trading strategy, risk tolerance, and the assets you are trading. Different time frames offer different trade-offs, and there is no one-size-fits-all answer. Here are some considerations for selecting a time frame:

1-Minute (or Less) Time Frame:
- Very short time frames like 1 minute are often used by high-frequency traders (HFT) and algorithmic trading systems that execute many trades in a short time. They require low latency and advanced infrastructure.
- High-frequency trading is associated with increased transaction costs, such as trading fees and potential slippage, and may not be suitable for all traders.
- The price action can be more erratic, making it challenging to identify meaningful trends, and there's a risk of "noise" in the data.
  
Shorter Time Frames (5-Minute, 15-Minute, 30-Minute):
- These time frames are often used by day traders and short-term momentum traders.
- They allow for more trades throughout the day but still require careful risk management due to the rapid price changes.

Longer Time Frames (1-Hour, 4-Hour, Daily):
- Longer time frames are generally used by swing traders and medium-term traders.
- Trends and patterns on these time frames tend to be more reliable, with less "noise" compared to shorter time frames.
- Trades occur less frequently, and holding positions for longer periods may require more patience.
- When choosing a time frame for a momentum trading bot, consider the following factors:

Trading Style: Your trading style and objectives matter. If you want to capitalize on very short-term price movements, you might choose a shorter time frame. For longer-term trades, consider longer time frames.

Asset Volatility: The volatility of the asset you're trading is crucial. Highly volatile assets might require shorter time frames to capture momentum, while less volatile assets can be traded on longer time frames.

Risk Management: Shorter time frames often require tighter stop-loss orders and more frequent monitoring, so you need a robust risk management strategy.

Technical Analysis: The time frame you choose should align with your technical analysis. Different time frames may show different patterns and signals.

Infrastructure: High-frequency trading often demands low-latency infrastructure, including high-speed internet connections and powerful servers.

Backtesting: Use backtesting to evaluate the performance of your trading strategy on historical data across various time frames. This can help you determine which time frame works best for your specific strategy.